In [1]:
import torch
import numpy as np
import pandas as pd
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
from numpy.linalg import norm

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
# 設定語意模型
def get_semantic_model():
    model = ORTModelForFeatureExtraction.from_pretrained('clw8998/semantic_model-for-EE5327701')
    tokenizer = AutoTokenizer.from_pretrained('clw8998/semantic_model-for-EE5327701')
    return model, tokenizer

In [3]:
# 定義 MOMO 爬蟲函數
def scrapy_MOMO(keyword='鞋櫃', page=5, max_item_num=30):
    headers = {
        "Referer": "https://m.momoshop.com.tw/",
        'User-Agent': 'Mozilla/5.0'
    }
    df = pd.DataFrame(columns=['Title', 'Price'])
    item_number = 0
    for index in range(1, page + 1):
        url = f'https://m.momoshop.com.tw/search.momo?curPage={index}&searchKeyword={keyword}'
        resp = requests.get(url, headers=headers)
        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, "lxml")
            for item in soup.select('li.goodsItemLi > a'):
                title = str(item["title"]).title()
                price_element = item.find('b', {'class': 'price'})
                price = price_element.text if price_element else '0'
                df = pd.concat([df, pd.DataFrame({'Title': title, 'Price': price}, index=[0])], ignore_index=True)
        df = df.drop_duplicates(ignore_index=True)
        if df.shape[0] >= max_item_num:
            break
    return df[:max_item_num]

In [4]:
# 讀取搜尋詞和商品名稱
def load_search_terms(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        search_terms = f.read().splitlines()
    return search_terms

def load_product_data(filename):
    df = pd.read_csv(filename)
    return df['product_name'].head(30).tolist()

# 嵌入向量計算
def inference(tokenizer, model, texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    embeddings = model(**inputs).last_hidden_state.mean(dim=1).detach().numpy()
    return embeddings

# 計算餘弦相似度
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

In [5]:
student_id = 'M11207321'
threshold = 0.9
# 加載模型
model, tokenizer = get_semantic_model()

# 讀取搜尋詞和產品名稱
search_terms = load_search_terms(f'{student_id}_queries.txt')

In [6]:
for search_term in search_terms:
    # 從 CSV 讀取商品名稱
    product_names = load_product_data(f'./results/{student_id}_{search_term}.csv')

    # 顯示產品名稱
    print(f"來自 {student_id}_{search_term}.csv 的前 30 筆商品：")
    for i, name in enumerate(product_names, 1):
        print(f"{i}. {name}")

    # MOMO 爬取商品資料
    momo_products = scrapy_MOMO(search_term)
    momo_titles = momo_products['Title'].tolist()

    # 顯示 MOMO 商品名稱
    print(f"\n來自 MOMO 的前 30 筆商品：")
    for i, title in enumerate(momo_titles, 1):
        print(f"{i}. {title}")

    # 嵌入 MOMO 商品名稱
    momo_embeddings = inference(tokenizer, model, momo_titles)
    product_embeddings = inference(tokenizer, model, product_names)

    # 匹配商品並生成 DataFrame
    all_products = []

    for momo_title, momo_embedding in tqdm(zip(momo_titles, momo_embeddings), desc="匹配中"):
        max_similarity = 0
        best_match = None
        for product_name, product_embedding in zip(product_names, product_embeddings):
            similarity = cosine_similarity(momo_embedding, product_embedding)
            if similarity > max_similarity:
                max_similarity = similarity
                best_match = product_name
        if max_similarity >= threshold:
            all_products.append((momo_title, best_match, max_similarity))
        else:
            all_products.append((momo_title, None, None))  # 未匹配時 Product Title 和 Similarity 為空

    # 將匹配的商品放在最上面，未匹配的商品在下面
    matched_products = [item for item in all_products if item[1] is not None]
    unmatched_products = [item for item in all_products if item[1] is None]

    # 統一 DataFrame 結構
    final_df = pd.DataFrame(matched_products + unmatched_products, columns=['MOMO Title', 'Product Title', 'Similarity'])

    # 顯示結果
    display(final_df)

來自 M11207321_筆電.csv 的前 30 筆商品：
1. ESENSE 逸盛 E-C11 冷光五風扇筆電散熱墊 22-WNF011BK 380 x 290 x 32mm, 黑色
2. SOYMIX 10X10 皮革筆電包, 黑色的
3. Dest 筆記型電腦包, 黑色的
4. Home Planet 可調節鋁合金筆電增高架, 銀色, 1個
5. Dist Jelly Coco 筆記本電腦包 + Pin Button Set 2, 筆電包(海軍藍)+別針(A+B)
6. 7段高度調節筆電支架, 白色的
7. LG Gram真皮筆記型電腦包, Brown+Beige
8. 筆電支架, 灰色的
9. Kording 多口袋筆記本電腦包 KD-19, 灰色的
10. DEST JELLYCOCO 筆記型電腦包 + 夾扣 2件套裝, 筆電包(黑色)+別針(A+B)
11. Home Planet 升降摺疊筆電桌, 橡木色
12. Home Planet 鋁製筆記型電腦支架 MR-MS-NB01, 銀色
13. Macbook Pro 13吋通用矽膠鍵盤套, 黑色的, 1個
14. Kavalan 折疊式鋁合金筆電 平板支架 大 95-KAV01595-KAV015, 1個
15. Home Planet 折疊式筆電支架+手機支架組, 黑色, 1組
16. VIEWSEE MacBook 透明保護殼, 透明
17. danny house 羅技k380無線鍵盤膜, 1個
18. New'N MacBook 筆記型電腦包 CP58, 深灰色
19. 筆記型電腦收納包支架+配件收納包, 尺寸 14.1 至 15.4
20. SolRoom 筆記型電腦手提袋+肩帶, 天藍色
21. Home Planet 鋁製筆記型電腦支架, 鈦黑色
22. Home Planet 筆記型電腦金屬散熱支架, 黑色的
23. ROMANE Brunch Brother Pom Pom毛圈平板電腦包, 粉色
24. MUZIK TIGER 平板電腦包 12-14吋適用, 粉紅色
25. iamdue 筆記型電腦包, 灰色的
26. 動物園 iPad 包 11, 2沙漠狐灰色
27. Healing Shield Galaxy Book 4 Pro 16 低反液晶+頂部+底部+觸控板保護膜套裝N

匹配中: 30it [00:00, 3331.10it/s]


,MOMO Title,Product Title,Similarity
0,【Acer 宏碁】16吋14代I9 Rtx4060電競筆電(Predator/Phn16-7...,None,None
1,【Asus 華碩】福利品 11.6吋N4020翻轉觸控筆電(C214Ma Chromeboo...,None,None
2,【Acer 宏碁】16吋14代I9 Rtx4070電競筆電-黑神話悟空適用(Predator...,None,None
3,【Asus 華碩】11.6吋N4500文書輕薄筆電(E210/N4500/4G/64G/W1...,None,None
4,【Hp 惠普】15吋 Processor N100 輕薄四核心學生文書筆電(超品15/4G/...,None,None
5,【Acer 宏碁】筆電包/滑鼠組★15.6吋N4500輕薄筆電(Aspire/Al15-31...,None,None
6,【Acer 宏碁】15.6吋N4500輕薄筆電(Aspire/Al15-31P-C4L0/N...,None,None
7,【Hp 惠普】15吋N6000輕薄四核心學生筆電-極地白(超品15/15S-Fq3079Tu...,None,None
8,【Asus】筆電包/滑鼠組★15.6吋8G輕薄文書筆電(E510Ka/N4500/8G/12...,None,None
9,【Asus 華碩】14吋N4500輕薄筆電(E410Ka/N4500/8G/512G Ssd...,None,None


來自 M11207321_衣服.csv 的前 30 筆商品：
1. United Athle 素面短T #5001, 深卡其
2. planet re:maker 中性款露營涼感短袖T恤, 海軍藍
3. CARET 男款運動短褲
4. CARET 女款機能型內搭褲
5. Apinar 捲袖短袖襯衫
6. planet re:maker 中性款北極熊露營圖案涼感短袖T恤, 白色
7. NIKE 男款 PARK 20 SWOOSH 短袖 T恤
8. UNI.J 女士 Werin V領開叉短袖 T恤
9. Bonaraon 華夫格分層短袖 T 卹, 03灰色
10. PANG PANG SHOP 短袖連身蛋糕裙, 深灰色
11. PAPERMENTS 女款夏季修身針織T恤
12. Goodiff Urban 字母印花紮染寬鬆版型短袖 T 卹 + 短褲套裝
13. V領鏤空蕾絲襯衫
14. JAL BBA JEAN 女款涼感鬆緊長褲
15. LUNAGIRL Resent 格紋襯衫
16. CARET 男款運動短褲
17. 女士日常寬鬆版型鑲邊牛仔褲
18. planet re:maker 中性款北極熊露營圖案涼感短袖T恤, 深灰色
19. Queen Ease 女女孩刺繡寬鬆版型綁帶牛仔褲
20. JAL BBA JEAN 女款夏季彈性喇叭褲
21. ARMEDES 男款機能涼感短袖T恤 AR-191
22. CARET 男士修身涼感短褲
23. IMMONG 女款圓領層次感側綁帶A字上衣
24. ARMEDES 男款機能性乾爽短袖T恤 AR-192
25. POPLAR NC 有領寬鬆連身裙, 灰色的
26. lemon soul 短袖寬鬆牛仔連衣裙, 淺藍
27. BABY&I 女童鯊魚寶寶圖案3角內褲組 5入
28. ARMEDES 男款機能性乾爽涼感T恤 AR-194
29. IMMONG 大尺碼V領褶皺亞麻連身裙, 橘子
30. ARMEDES 男款乾爽運動短褲 AR-82

來自 MOMO 的前 30 筆商品：
1. 【Ob 嚴選】小V臉V領壓褶荷葉袖長版上衣 《Ab17119》
2. 【D.Studio】韓版寬鬆百搭圓領短袖上衣(短袖T恤 女裝 顯瘦上衣 衣服 寬鬆上衣 素T T187)
3. 【Ob 嚴選】優雅質感不易皺雪紡短袖上衣(襯衫/Ol/兩件式/透

匹配中: 30it [00:00, 3748.04it/s]


...

匹配中: 30it [00:00, 3744.69it/s]


,MOMO Title,Product Title,Similarity
0,【Csd 中衛】醫療口罩 成人立體3Dv 多色可選(30片/盒),"CSD 中衛 成人一級醫療口罩 17.5 x 9.5cm, 薄藍, 50片, 3盒",0.906363
1,【Csd 中衛】第一級醫療口罩-藍色/綠色/粉色 3盒入(50片/盒),"CSD 中衛 成人一級醫療口罩 17.5 x 9.5cm, 薄藍, 50片, 2盒",0.960769
2,【Csd 中衛】中衛醫療口罩-成人平面-多色可選(50入/盒.30入/盒),"CSD 中衛 成人一級醫療口罩 17.5 x 9.5cm, 薄藍, 50片, 1盒",0.952783
3,【Csd 中衛】中衛醫療口罩-成人平面-酷黑(100入/盒),"CSD 中衛 成人一級醫療口罩 17.5 x 9.5cm, 薄藍, 50片, 1盒",0.927363
4,【Motex 摩戴舒】雙鋼印 成人醫療口罩 多色任選 50入/盒 專品藥局(雙鋼印),"MOTEX 摩戴舒 平面醫用口罩 外耳 未滅菌 L, 原色黑, 50入, 2盒",0.922126
5,【德冠Dg】3D立體成人醫療口罩 X1盒(50入/盒 親水層光滑不起毛),None,NaN
6,【宏瑋】滿版口罩 耳帶撞色 50入 莫蘭迪色 醫用口罩 雙鋼印 醫療用口罩 平面口罩 Md ...,None,NaN
7,【醫強】成人3D立體醫用口罩 X1盒(30入/盒 極舒適 極輕量 超無感),None,NaN
8,【淨新】淨新醫療口罩 50枚入(平面口罩 淨新口罩 醫療口罩 成人口罩 兒童口罩 醫用口罩 ...,None,NaN
9,【艾爾絲】成人平面醫療口罩3盒組50片/入(Light輕時尚 多款任選 官方直營 網路限定色),None,NaN


來自 M11207321_吹風機.csv 的前 30 筆商品：
1. Kinyo 吹風機 800W, KH-193, 米色
2. Kinyo 輕巧負離子吹風機 1000W 140 x 70 x 220mm, KH-9505, 白色
3. Kinyo 吹風機 1200W 15 x 25 x 7.3cm 455g, KH-9201, 灰色
4. PHILIPS 飛利浦 ThermoProtect吹風機 2200W, HP8232/00, 白色 金色
5. PHILIPS 飛利浦 5000系列智能護髮負離子吹風機 2200W, BHD538/29, 黑色 玫瑰金
6. Kinyo 輕巧水潤負離子吹風機 1000W 110V 220 x 140 x 70mm 370g, KH-9505, 黑色
7. PHILIPS 飛利浦 負離子恆溫吹風機 BHD500/09 2200W, 白色
8. 達新牌 雙電壓折疊吹風機 1400W, FD-176
9. Kinyo 陶瓷負離子吹風機 1200W, KH-9201, 粉色
10. Kinyo 雙電壓折疊式吹風機, KH-192, 白色
11. Happy+ Superstorm 專業吹風機 BY-596 1600~1800W, 單色
12. VS 沙宣 強風吹風機, VSD120K, 黑色玫瑰金
13. Kinyo 時尚輕巧吹風機 750W 180 x 190 x 75mm 300g, KH-7502, 棕色
14. ShowSee 負離子吹風機 1800W, A1-W, 白色
15. patech 大風力吹風機 PD-H4300 2300W, 黑色
16. AIDEN 吹風機 WA-1800HD 1800W, 混色
17. UNIX 白金色吹風機 1700W, UN-A1690, 白金色
18. PHILIPS 飛利浦 負離子護髮吹風機 2100W, HP8230/00, 黑色
19. PHILIPS 飛利浦 護理吹風機 2100W, BHD340/19, 紫色
20. Panasonic Nano Care吹風機 EH-NA98, 粉色
21. 飛利浦基本護理吹風機 BHC010/19 1200W, 黑色
22. VS 沙宣 吹風機 VSD565K 2000W, 白玫瑰
23. VS 沙宣 強力玫瑰金吹風機2000W, VSD51

匹配中: 30it [00:00, 3747.14it/s]


,MOMO Title,Product Title,Similarity
0,【Panasonic 國際牌】奈米水離子吹風機(Eh-Na27-Pp),"Panasonic Nano Care吹風機 EH-NA98, 粉色",0.928913
1,【小米官方旗艦館】Xiaomi水離子吹風機 H500 白色,"Xiaomi 小米 負離子吹風機 H300, CMJ02ZHM, 白色的",0.944132
2,【Panasonic 國際牌】奈米水離子吹風機-粉金(Eh-Na9G-Pn),"Panasonic Nano Care吹風機 EH-NA98, 粉色",0.911152
3,【Kinyo】超輕量美型吹風機(Kh-196),"Kinyo 吹風機 800W, KH-193, 米色",0.978290
4,【Vs 沙宣】千萬負離子柔亮護髮吹風機(Vs1820Bw),"VS 沙宣 吹風機 VSD565K 2000W, 白玫瑰",0.927609
5,【Philips 飛利浦】沙龍級護髮負離子吹風機-霧藍黑(Bhd518/01),"PHILIPS 飛利浦 負離子恆溫吹風機 BHD500/09 2200W, 白色",0.965447
6,【Panasonic 國際牌】奈米水離子吹風機(Eh-Na27-W),"Panasonic Nano Care吹風機 EH-NA98, 粉色",0.925216
7,【小米官方旗艦館】小米負離子吹風機 H300 白色,"Xiaomi 小米 負離子吹風機 H300, CMJ02ZHM, 白色的",0.950315
8,【Panasonic 國際牌】外置負離子速乾型吹風機(Eh-Ne29-K),"Panasonic Nano Care吹風機 EH-NA98, 粉色",0.904840
9,【達新牌】光觸媒抗菌8000萬超水潤負離子吹風機(Fd-808),"達新牌 雙電壓折疊吹風機 1400W, FD-176",0.945644
